In [1]:
#optional installations: 
#!pip install yfinance --upgrade --no-cache-dir
#!pip3 install pandas_datareader


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# ___library_import_statements___
import pandas as pd

# for pandas_datareader, otherwise it might have issues, sometimes there is some version mismatch
pd.core.common.is_list_like = pd.api.types.is_list_like

# make pandas to print dataframes nicely
pd.set_option('expand_frame_repr', False)  

import pandas_datareader.data as web
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time

#newest yahoo API 
import yfinance as yahoo_finance

#optional 
#yahoo_finance.pdr_override()

%matplotlib inline


In [2]:
# ___variables___
#ticker = 'AAPL'
#ticker = 'TSLA'
#ticker = 'GOOGL'
#ticker = 'NFLX'
#ticker = 'GOOG'
ticker = 'BIDU'

start_time = datetime.datetime(2017, 1, 1)
#end_time = datetime.datetime(2019, 1, 20)
end_time = datetime.datetime.now().date().isoformat()         # today


In [3]:
# yahoo gives only daily historical data
connected = False
while not connected:
    try:
        df = web.get_data_yahoo(ticker, start=start_time, end=end_time)
        connected = True
        print('connected to yahoo')
    except Exception as e:
        print("type error: " + str(e))
        time.sleep( 5 )
        pass   

# use numerical integer index instead of date    
df = df.reset_index()
print(df.head(5))


connected to yahoo
        Date        High         Low        Open       Close   Volume   Adj Close
0 2017-01-03  169.660004  165.820007  166.300003  168.300003  2314700  168.300003
1 2017-01-04  173.259995  168.300003  169.740005  171.960007  2033200  171.960007
2 2017-01-05  177.839996  172.500000  172.649994  177.470001  2733000  177.470001
3 2017-01-06  177.279999  173.250000  176.279999  176.380005  2254700  176.380005
4 2017-01-09  179.720001  176.820007  178.000000  177.160004  1760400  177.160004


In [4]:
def bollinger_bands(df, n, m):
    # takes dataframe on input
    # n = smoothing length
    # m = number of standard deviations away from MA
    
    #typical price
    TP = (df['High'] + df['Low'] + df['Close']) / 3
    # but we will use Adj close instead for now, depends
    
    data = TP
    #data = df['Adj Close']
    
    # takes one column from dataframe
    B_MA = pd.Series((data.rolling(n, min_periods=n).mean()), name='B_MA')
    sigma = data.rolling(n, min_periods=n).std() 
    
    BU = pd.Series((B_MA + m * sigma), name='BU')
    BL = pd.Series((B_MA - m * sigma), name='BL')
    
    df = df.join(B_MA)
    df = df.join(BU)
    df = df.join(BL)
    
    return df


In [5]:
n = 20   # datapoint rolling window
m = 2    # sigma width
df = bollinger_bands(df, 20, 2)


In [6]:
print(df.head())
print(df.tail())


        Date        High         Low        Open       Close   Volume   Adj Close  B_MA  BU  BL
0 2017-01-03  169.660004  165.820007  166.300003  168.300003  2314700  168.300003   NaN NaN NaN
1 2017-01-04  173.259995  168.300003  169.740005  171.960007  2033200  171.960007   NaN NaN NaN
2 2017-01-05  177.839996  172.500000  172.649994  177.470001  2733000  177.470001   NaN NaN NaN
3 2017-01-06  177.279999  173.250000  176.279999  176.380005  2254700  176.380005   NaN NaN NaN
4 2017-01-09  179.720001  176.820007  178.000000  177.160004  1760400  177.160004   NaN NaN NaN
           Date        High         Low        Open       Close   Volume   Adj Close        B_MA          BU          BL
1422 2022-08-26  155.479996  147.470001  154.270004  147.710007  3317600  147.710007  136.723334  147.450815  125.995853
1423 2022-08-29  151.669998  147.179993  148.300003  147.320007  2335800  147.320007  137.475501  149.352480  125.598522
1424 2022-08-30  148.990005  134.710007  147.949997  137.6900

In [ ]:
# plot price
plt.figure(figsize=(15,5))
plt.plot(df['Date'], df['Adj Close'])
plt.title('Price chart (Adj Close) ' + str(ticker))
plt.show()

# plot correspondingRSI values and significant levels
plt.figure(figsize=(15,5))
plt.title('Bollinger Bands chart ' + str(ticker))
plt.plot(df['Date'], df['Adj Close'])
plt.plot(df['Date'], df['BU'], alpha=0.3)
plt.plot(df['Date'], df['BL'], alpha=0.3)
plt.plot(df['Date'], df['B_MA'], alpha=0.3)
plt.fill_between(df['Date'], df['BU'], df['BL'], color='grey', alpha=0.1)

plt.show()
